In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler

# Q1: What is the mathematical formula for a linear SVM?
# A linear SVM aims to find a hyperplane that separates the data into two classes. The decision function can be written as:
# f(x) = w.T * x + b
# where w is the weight vector, x is the input feature vector, and b is the bias term.

# Q2: What is the objective function of a linear SVM?
# The objective function for a linear SVM is to minimize the following:
# min (1/2) * ||w||^2 + C * sum(ξ_i)
# subject to y_i * (w.T * x_i + b) >= 1 - ξ_i for all i and ξ_i >= 0
# where ξ_i are the slack variables that allow for misclassifications, and C is the regularization parameter that controls the trade-off between maximizing the margin and minimizing the classification error.

# Q3: What is the kernel trick in SVM?
# The kernel trick allows SVMs to create nonlinear decision boundaries by implicitly mapping the input features into a higher-dimensional space. Instead of computing the mapping explicitly, the kernel trick computes the inner product of the mapped features directly using a kernel function. Common kernels include the polynomial kernel and the radial basis function (RBF) kernel.

# Q4: What is the role of support vectors in SVM? Explain with an example.
# Support vectors are the data points that lie closest to the decision boundary (hyperplane). They are the critical elements of the training set because they define the position and orientation of the hyperplane. The SVM algorithm optimizes the margin between the support vectors of the two classes.

# Example:
# Consider a binary classification problem with two features. The support vectors are the points that lie on the margins or violate them slightly (in the case of soft margin SVM). The hyperplane is adjusted based on these support vectors, ensuring the maximum margin between the classes.

# Q5: Illustrate with examples and graphs of Hyperplane, Marginal plane, Soft margin, and Hard margin in SVM.
# We will use synthetic data to illustrate these concepts.

# Generate synthetic data
from sklearn.datasets import make_blobs

X, y = make_blobs(n_samples=100, centers=2, random_state=42, cluster_std=1.0)
y = np.where(y == 0, -1, 1)  # Convert labels to -1 and 1 for SVM

plt.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm')
plt.title("Synthetic Data for SVM Illustration")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.show()

# Train a hard margin SVM
svm_hard = SVC(kernel='linear', C=1e5)
svm_hard.fit(X, y)

# Plot the decision boundary and support vectors
def plot_svm(svm, X, y):
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm')
    ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    # Create grid to evaluate model
    xx, yy = np.meshgrid(np.linspace(xlim[0], xlim[1], 50),
                         np.linspace(ylim[0], ylim[1], 50))
    xy = np.vstack([xx.ravel(), yy.ravel()]).T
    Z = svm.decision_function(xy).reshape(xx.shape)

    # Plot decision boundary and margins
    ax.contour(xx, yy, Z, colors='k', levels=[-1, 0, 1], alpha=0.5,
               linestyles=['--', '-', '--'])
    ax.scatter(svm.support_vectors_[:, 0], svm.support_vectors_[:, 1], s=100,
               linewidth=1, facecolors='none', edgecolors='k')
    plt.xlabel("Feature 1")
    plt.ylabel("Feature 2")
    plt.title("SVM Decision Boundary and Margins")
    plt.show()

plot_svm(svm_hard, X, y)

# Train a soft margin SVM
svm_soft = SVC(kernel='linear', C=1.0)
svm_soft.fit(X, y)
plot_svm(svm_soft, X, y)

# Q6: SVM Implementation through Iris dataset
# Load the iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Use only two classes for binary classification
X = X[y != 2]
y = y[y != 2]

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a linear SVM classifier using scikit-learn
svm_clf = SVC(kernel='linear', C=1.0)
svm_clf.fit(X_train, y_train)

# Predict the labels for the testing set
y_pred = svm_clf.predict(X_test)

# Compute the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of scikit-learn SVM: {accuracy:.2f}")

# Plot the decision boundaries using two features
def plot_decision_boundaries(X, y, model, title):
    h = .02  # step size in the mesh
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.8)
    plt.scatter(X[:, 0], X[:, 1], c=y, edgecolors='k', marker='o', s=50, cmap=plt.cm.coolwarm)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.title(title)
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.show()

# Plot decision boundaries
plot_decision_boundaries(X_test[:, :2], y_test, svm_clf, "SVM Decision Boundaries")

# Try different values of the regularization parameter C
C_values = [0.01, 0.1, 1, 10, 100]
accuracies = []

for C in C_values:
    svm_clf = SVC(kernel='linear', C=C)
    svm_clf.fit(X_train, y_train)
    y_pred = svm_clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    accuracies.append(acc)
    print(f"Accuracy with C={C}: {acc:.2f}")

# Plot the effect of C on accuracy
plt.plot(C_values, accuracies, marker='o')
plt.xscale('log')
plt.xlabel('Regularization parameter C')
plt.ylabel('Accuracy')
plt.title('Effect of C on SVM Accuracy')
plt.show()

# Bonus Task: Implement a linear SVM classifier from scratch
class LinearSVM:
    def __init__(self, C=1.0, learning_rate=0.001, n_iters=1000):
        self.C = C
        self.lr = learning_rate
        self.n_iters = n_iters
        self.w = None
        self.b = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        y_ = np.where(y <= 0, -1, 1)

        self.w = np.zeros(n_features)
        self.b = 0

        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                condition = y_[idx] * (np.dot(x_i, self.w) - self.b) >= 1
                if condition:
                    self.w -= self.lr * (2 * self.w / self.n_iters)
                else:
                    self.w -= self.lr * (2 * self.w / self.n_iters - np.dot(x_i, y_[idx]))
                    self.b -= self.lr * y_[idx]

    def predict(self, X):
        approx = np.dot(X, self.w) - self.b
        return np.sign(approx)

# Train the custom SVM
svm_scratch = LinearSVM(C=1.0, learning_rate=0.001, n_iters=1000)
svm_scratch.fit(X_train, y_train)
y_pred_scratch = svm_scratch.predict(X_test)

# Compute accuracy
accuracy_scratch = accuracy_score(y_test, y_pred_scratch)
print(f"Accuracy of custom SVM: {accuracy_scratch:.2f}")

# Compare with scikit-learn implementation
print(f"Accuracy of scikit-learn SVM: {accuracy:.2f}")
print(f"Accuracy of custom SVM: {accuracy_scratch:.2f}")
